In [1]:
import pandas as pd
from novel_risk_assistant_only_select_choice import get_risk_3_type_to_risk_cat_dict
import os
import json

dir_path = os.getcwd()
risk_data_df_path = os.path.join(dir_path, "data/20240811_ALL_risks.xlsx")

risk_3_type_to_risk_cat_dict = get_risk_3_type_to_risk_cat_dict(risk_data_df_path)

In [2]:
# input_sentence_column = "input_risk_name"
# df_path = "/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/src/cro_rmi_improvement_feature/find_similar_risk/comparison_result_df_input_name_with_risk_cat.pkl"
# input_sentence_column = "input_risk_desc"
# df_path = "/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/src/cro_rmi_improvement_feature/find_similar_risk/comparison_result_df_input_desc_with_risk_cat.pkl"
input_sentence_column = "input_risk_name_and_desc"
df_path = "/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/src/cro_rmi_improvement_feature/find_similar_risk/comparison_result_df_input_name_and_desc_with_risk_cat.pkl"
df = pd.read_pickle(df_path)
df = df.iloc[:, 1:]
df["candidate_sentences"] = df["candidate_sentences"].apply(str)

print(type(df["sorted_similar_sentences"][0]))
df.head()

<class 'list'>


,target_sentence,candidate_sentences,embedded_model,distance_method,sorted_similar_sentences,sorted_similar_sentences_indices,target_order_in_sorted_similar_sentences,customer_risk_cat,target_risk_cat
0,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",all-MiniLM-L6-v2,cosine,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119,Operational risk,Operational risk
1,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",all-MiniLM-L6-v2,euclidean,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119,Operational risk,Operational risk
2,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",openai-small,cosine,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1,Operational risk,Operational risk
3,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",openai-small,euclidean,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1,Operational risk,Operational risk
4,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",openai-large,cosine,"[เครื่องจักรหรืออุปกรณ์สูญหาย, เครื่องจักรหรือ...","[9, 8, 4, 60, 57, 15, 61, 48, 67, 76, 13, 43, ...",2,Operational risk,Operational risk


In [3]:
# loop each row in df
df["filter_sorted_similar_sentences"] = None
df["target_order_in_filter_sorted_similar_sentences"] = None
for index, row in df.iterrows():
    new_sorted_similar_sentences = []
    target_risk_cat = row["target_risk_cat"]
    target_sentence = row["target_sentence"]
    # print(f"target_risk_cat: {target_risk_cat},{type(target_risk_cat)}")
    for sentence in row["sorted_similar_sentences"]:
        # print(sentence)
        current_risk_cat = risk_3_type_to_risk_cat_dict[sentence]
        if current_risk_cat == target_risk_cat:
            new_sorted_similar_sentences.append(sentence)
    # update new_sorted_similar_sentences to column filter_sorted_similar_sentences
    try:
        df.at[index, "filter_sorted_similar_sentences"] = new_sorted_similar_sentences
    except ValueError:
        print(f"ValueError: {new_sorted_similar_sentences}")
        print(f"target_risk_cat: {target_risk_cat}")
        print(f"index: {index}")
        print(f"row: {row}")
        raise
    # then update target_order_in_filter_sorted_similar_sentences
    target_order_in_filter_sorted_similar_sentences = (
        new_sorted_similar_sentences.index(target_sentence) + 1
    )
    df.at[index, "target_order_in_filter_sorted_similar_sentences"] = (
        target_order_in_filter_sorted_similar_sentences
    )

In [4]:
# is_same_risk_cat column check target_risk_cat == customer_risk_cat
df["is_same_risk_cat"] = df.apply(
    lambda x: 1 if x["target_risk_cat"] == x["customer_risk_cat"] else 0, axis=1
)
# show df that has is_same_risk_cat == 0
df[df["is_same_risk_cat"] == 1]

,target_sentence,candidate_sentences,embedded_model,distance_method,sorted_similar_sentences,sorted_similar_sentences_indices,target_order_in_sorted_similar_sentences,customer_risk_cat,target_risk_cat,filter_sorted_similar_sentences,target_order_in_filter_sorted_similar_sentences,is_same_risk_cat
0,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",all-MiniLM-L6-v2,cosine,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119,Operational risk,Operational risk,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,53,1
1,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",all-MiniLM-L6-v2,euclidean,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119,Operational risk,Operational risk,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,53,1
2,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",openai-small,cosine,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1,Operational risk,Operational risk,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...",1,1
3,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",openai-small,euclidean,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1,Operational risk,Operational risk,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...",1,1
4,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ'...",openai-large,cosine,"[เครื่องจักรหรืออุปกรณ์สูญหาย, เครื่องจักรหรือ...","[9, 8, 4, 60, 57, 15, 61, 48, 67, 76, 13, 43, ...",2,Operational risk,Operational risk,"[เครื่องจักรหรืออุปกรณ์สูญหาย, เครื่องจักรหรือ...",2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2539,การถูกโจมตีแบบฟิชชิ่ง (Phishing) หรือ การโจมตี...,['การขาดแคลนวัตถุดิบ \nการขาดแคลนหรือข้อบกพร่อ...,openai-small,euclidean,[การถูกโจมตีทางไซเบอร์จากการนำเทคโนโลยี Blockc...,"[90, 83, 87, 72, 73, 74, 70, 84, 71, 76, 86, 9...",9,Operational risk,Operational risk,[การถูกโจมตีทางไซเบอร์จากการนำเทคโนโลยี Blockc...,9,1
2540,การถูกโจมตีแบบฟิชชิ่ง (Phishing) หรือ การโจมตี...,['การขาดแคลนวัตถุดิบ \nการขาดแคลนหรือข้อบกพร่อ...,openai-large,cosine,[การถูกโจมตีด้วยวิธี Brute Force\nความเสี่ยงจา...,"[76, 75, 83, 80, 84, 92, 90, 74, 93, 71, 78, 8...",10,Operational risk,Operational risk,[การถูกโจมตีด้วยวิธี Brute Force\nความเสี่ยงจา...,10,1
2541,การถูกโจมตีแบบฟิชชิ่ง (Phishing) หรือ การโจมตี...,['การขาดแคลนวัตถุดิบ \nการขาดแคลนหรือข้อบกพร่อ...,openai-large,euclidean,[การถูกโจมตีด้วยวิธี Brute Force\nความเสี่ยงจา...,"[76, 75, 83, 80, 84, 92, 90, 74, 93, 71, 78, 8...",10,Operational risk,Operational risk,[การถูกโจมตีด้วยวิธี Brute Force\nความเสี่ยงจา...,10,1
2542,การถูกโจมตีแบบฟิชชิ่ง (Phishing) หรือ การโจมตี...,['การขาดแคลนวัตถุดิบ \nการขาดแคลนหรือข้อบกพร่อ...,gemini-exp-03-07,cosine,[การถูกโจมตีทางไซเบอร์จากการนำเทคโนโลยี Blockc...,"[90, 78, 80, 76, 84, 75, 70, 83, 96, 73, 91, 8...",16,Operational risk,Operational risk,[การถูกโจมตีทางไซเบอร์จากการนำเทคโนโลยี Blockc...,16,1


In [5]:
# loop 5,10,15,20,25
for i in range(5, 30, 5):
    # create a new column for each loop call top_{i}
    # if target_order_in_filter_sorted_similar_sentences is <= i, then 1 else 0
    df[f"top_{i}"] = df["target_order_in_filter_sorted_similar_sentences"].apply(
        lambda x: 1 if x <= i else 0
    )

In [6]:
df.candidate_sentences.unique()

array(["['การขาดแคลนวัตถุดิบ ', 'วัตถุดิบไม่ได้คุณภาพ', 'อำนาจต่อรองกับซัพพลายเออร์ต่ำ', 'การขาดความร่วมมือภายในองค์กร', 'กระบวนการทำงานไม่มีประสิทธิภาพ', 'การขาดประสิทธิภาพในการจัดจ้างงานภายนอก (outsourcing)', 'ข้อผิดพลาดทางบัญชี', 'ความล้มเหลวในการดําเนินธุรกิจใหม่', 'เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย', 'เครื่องจักรหรืออุปกรณ์สูญหาย', 'ยานพาหนะขัดข้องหรือเสียหาย', 'ยานพาหนะสูญหาย', 'โครงสร้างพื้นฐานขัดข้องหรือเสียหาย', 'ผลิตภัณฑ์ไม่ได้คุณภาพ', 'ความไม่ปลอดภัยของผลิตภัณฑ์', 'การให้บริการไม่ได้คุณภาพ', 'สินค้าคงคลังน้อยเกินไป', 'สินค้าคงคลังมากเกินไป', 'สินค้าคงคลังล้าสมัย', 'สินค้าคงคลังหมดอายุ', 'สินค้าคงคลังเสียหาย', 'สินค้าคงคลังสูญหาย', 'การจัดส่งสินค้าผิด', 'สินค้าเสียหายระหว่างการจัดส่ง', 'สินค้าสูญหายระหว่างการจัดส่ง', 'ไม่สามารถจัดส่งสินค้าได้', 'การจัดส่งสินค้าล่าช้า', 'ผลิตภัณฑ์ไม่สามารถแข่งขันได้', 'การให้บริการที่ไม่สามารถแข่งขันได้', 'การตั้งราคาที่ไม่เหมาะสม', 'การตลาดที่ไม่มีประสิทธิภาพ', 'ช่องทางการจัดจําหน่ายแบบดั้งเดิมที่ไม่สามารถแข่งขันได้', 'ช่องทางการจัดจํา

In [7]:
df.candidate_sentences.unique()
candidate_sentences_dict_map = {
    df.candidate_sentences.unique()[0]: "target_risk_name",
    df.candidate_sentences.unique()[1]: "target_risk_desc",
    df.candidate_sentences.unique()[2]: "target_risk_name_and_desc",
}
df["candidate_sentences"] = df["candidate_sentences"].map(candidate_sentences_dict_map)

In [8]:
selected_df = df.copy()
selected_df = selected_df[selected_df["is_same_risk_cat"] == 1]
column_list = ["embedded_model", "distance_method", "candidate_sentences"]
# Group by type columns and sum the scores
result = (
    selected_df.groupby(column_list)
    .agg(
        {
            "top_5": "sum",
            "top_10": "sum",
            "top_15": "sum",
            "top_20": "sum",
            "top_25": "sum",
        }
    )
    .reset_index()
)

# Sort the result if needed (optional)
result = result.sort_values(column_list)
# result do the percentage
for i in range(5, 26, 5):
    result[f"top_{i}_percentage"] = (result[f"top_{i}"] / 84 * 100).round(2)
# add this on left input_sentence_column
result["input_sentence_column"] = input_sentence_column
# make input_sentence_column to first column and the rest to the right
result = result[
    ["input_sentence_column"]
    + column_list
    + [f"top_{i}_percentage" for i in range(5, 26, 5)]
]
result

,input_sentence_column,embedded_model,distance_method,candidate_sentences,top_5_percentage,top_10_percentage,top_15_percentage,top_20_percentage,top_25_percentage
0,input_risk_name_and_desc,all-MiniLM-L6-v2,cosine,target_risk_desc,14.29,20.24,23.81,36.90,39.29
1,input_risk_name_and_desc,all-MiniLM-L6-v2,cosine,target_risk_name,17.86,21.43,27.38,35.71,39.29
2,input_risk_name_and_desc,all-MiniLM-L6-v2,cosine,target_risk_name_and_desc,21.43,27.38,30.95,46.43,54.76
3,input_risk_name_and_desc,all-MiniLM-L6-v2,euclidean,target_risk_desc,14.29,20.24,23.81,36.90,39.29
4,input_risk_name_and_desc,all-MiniLM-L6-v2,euclidean,target_risk_name,17.86,21.43,27.38,35.71,39.29
5,input_risk_name_and_desc,all-MiniLM-L6-v2,euclidean,target_risk_name_and_desc,21.43,27.38,30.95,46.43,54.76
6,input_risk_name_and_desc,gemini-exp-03-07,cosine,target_risk_desc,67.86,79.76,84.52,88.10,89.29
7,input_risk_name_and_desc,gemini-exp-03-07,cosine,target_risk_name,69.05,80.95,84.52,88.10,89.29
8,input_risk_name_and_desc,gemini-exp-03-07,cosine,target_risk_name_and_desc,69.05,80.95,85.71,90.48,94.05
9,input_risk_name_and_desc,gemini-exp-03-07,euclidean,target_risk_desc,67.86,79.76,84.52,88.10,89.29


In [9]:
# save summary table
file_name = f"summary_table_{input_sentence_column}.xlsx"
# save to data folder
file_path = os.path.join(dir_path, "data", file_name)
result.to_excel(file_path)

In [10]:
table_1_path = "data/summary_table_input_risk_name.xlsx"
table_2_path = "data/summary_table_input_risk_desc.xlsx"
table_3_path = "data/summary_table_input_risk_name_and_desc.xlsx"
table_1 = pd.read_excel(table_1_path)
table_2 = pd.read_excel(table_2_path)
table_3 = pd.read_excel(table_3_path)
# stack rows table 1,2,3
all_table = pd.concat([table_1, table_2, table_3], axis=0).reset_index(drop=True)
# remove first column
all_table = all_table.iloc[:, 1:]
all_table

,input_sentence_column,embedded_model,distance_method,candidate_sentences,top_5_percentage,top_10_percentage,top_15_percentage,top_20_percentage,top_25_percentage
0,input_risk_name,all-MiniLM-L6-v2,cosine,target_risk_desc,13.10,21.43,21.43,29.76,32.14
1,input_risk_name,all-MiniLM-L6-v2,cosine,target_risk_name,14.29,19.05,25.00,27.38,34.52
2,input_risk_name,all-MiniLM-L6-v2,cosine,target_risk_name_and_desc,14.29,25.00,27.38,32.14,39.29
3,input_risk_name,all-MiniLM-L6-v2,euclidean,target_risk_desc,13.10,21.43,21.43,29.76,32.14
4,input_risk_name,all-MiniLM-L6-v2,euclidean,target_risk_name,14.29,19.05,25.00,27.38,34.52
...,...,...,...,...,...,...,...,...,...
67,input_risk_name_and_desc,openai-small,cosine,target_risk_name,51.19,61.90,71.43,72.62,72.62
68,input_risk_name_and_desc,openai-small,cosine,target_risk_name_and_desc,52.38,59.52,65.48,69.05,78.57
69,input_risk_name_and_desc,openai-small,euclidean,target_risk_desc,47.62,55.95,64.29,66.67,72.62
70,input_risk_name_and_desc,openai-small,euclidean,target_risk_name,51.19,61.90,71.43,72.62,72.62


In [11]:
# save all_table
file_name_path = "data/summary_table.xlsx"
all_table.to_excel(file_name_path)